In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd
import requests

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [20]:
links = []


In [27]:
restaurant = pd.DataFrame(columns=[
    "michelin_name", "kakao_name", "address", "latitude", "longitude",
    "category", "page", "price", "rating", "call_number"
])

In [77]:
restaurant['kakao_name'] = restaurant['kakao_name'].astype(str)
restaurant['address'] = restaurant['address'].astype(str)
restaurant['call_number'] = restaurant['call_number'].astype(str)

In [21]:
for i in range(1,7):
    driver.get("https://guide.michelin.com/kr/ko/selection/south-korea/restaurants/page/{}".format(i))
    time.sleep(2)

    # 현재 페이지에서 div들 찾기
    divs = driver.find_elements(By.XPATH, '/html/body/main/section[1]/div[1]/div/div[2]/div')

    for div in divs:
        a_tag = div.find_element(By.TAG_NAME, "a")
        href = a_tag.get_attribute("href")
        links.append(href)

In [28]:
for link in links:
    driver.get(link)
    time.sleep(2)

    name = driver.find_element(By.XPATH, '/html/body/main/div[4]/div[1]/div/div[1]/section/div/div[1]/div/h1').text

    address = driver.find_element(By.XPATH, '/html/body/main/div[4]/div[1]/div/div[1]/section/div/div[2]/div/div/div[1]/div[1]').text
    category = driver.find_element(By.XPATH, '/html/body/main/div[4]/div[1]/div/div[1]/section/div/div[2]/div/div/div[1]/div[2]').text

    iframe = driver.find_element(By.CSS_SELECTOR, "iframe[src*='google.com/maps']")
    iframe_src = iframe.get_attribute("src")
    if "q=" in iframe_src:
        coord_part = iframe_src.split("q=")[1].split("&")[0]
        coords = coord_part.split(",")
        if len(coords) == 2:
            latitude = coords[0].strip()
            longitude = coords[1].strip()
    parts = [p.strip() for p in category.split("·")]

    restaurant.loc[len(restaurant)] = {
        "michelin_name": name,
        "latitude": latitude,
        "longitude": longitude,
        "category": parts[1],
        "page": link,
        "price": parts[0]
    }

In [33]:
url = 'https://dapi.kakao.com/v2/local/search/keyword.json' 
headers = {"Authorization": "KakaoAK 633b071420476eea43525a6747c22e0c"}

In [ ]:
for i in range(len(restaurant)):
    row = restaurant.iloc[i]
    params = {
    'query': row['michelin_name'],  # 검색할 키워드 (예시: '카페')
    'x': row['longitude'],  # 중심점의 x 좌표
    'y': row['latitude'],  # 중심점의 y 좌표
    'radius': 50,  # 검색 반경 (미터 단위)
    }      
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:  # 요청이 성공한 경우
        data = response.json()  # JSON 형식으로 응답 데이터를 가져옴

        # 검색 결과가 존재하는지 확인
        if 'documents' in data and len(data["documents"]) > 0:
            doc = data["documents"][0]
            restaurant.loc[i, 'kakao_name'] = str(doc.get('place_name', ''))
            restaurant.loc[i, 'address'] = str(doc.get('road_address_name', ''))
            restaurant.loc[i, 'call_number'] = str(doc.get('phone', ''))
            restaurant.loc[i, 'kakao_id'] = doc.get('id', None)
        else:
            print(f"index={i}, name={row['michelin_name']}, 검색 결과 없음")  # 검색 결과가 없는 경우 메시지 출력
    else:
        print(f"요청 실패: index={i}, name={row['michelin_name']}", response.status_code)  # 요청 실패 시 상태 코드 출력

index=41, name=진진, 검색 결과 없음
index=60, name=오레노 라멘, 검색 결과 없음
index=68, name=명동 교자, 검색 결과 없음
index=78, name=삼청동 수제비, 검색 결과 없음
index=90, name=담택, 검색 결과 없음
index=105, name=스시 마츠모토, 검색 결과 없음
index=138, name=역전회관, 검색 결과 없음
index=176, name=구찌 오스테리아 다 마시모 보투라, 검색 결과 없음
index=182, name=오프닝, 검색 결과 없음
index=194, name=정육면체, 검색 결과 없음
index=198, name=으뜸 이로리바타, 검색 결과 없음
index=199, name=오부이용, 검색 결과 없음
index=211, name=미미 면가, 검색 결과 없음
index=223, name=레썽스, 검색 결과 없음
index=237, name=일드청담, 검색 결과 없음
index=244, name=르오뇽, 검색 결과 없음


In [79]:
for i in range(len(restaurant)):
    if restaurant.loc[i, 'price'] == '₩':
        restaurant.loc[i, 'price'] = '가격대 낮음'
    elif restaurant.loc[i, 'price'] == '₩₩':
        restaurant.loc[i, 'price'] = '가격대 보통'
    elif restaurant.loc[i, 'price'] == '₩₩₩':
        restaurant.loc[i, 'price'] = '가격대 약간 높음'
    elif restaurant.loc[i, 'price'] == '₩₩₩₩':
        restaurant.loc[i, 'price'] = '가격대 높음'

In [80]:
restaurant.loc[41, ['kakao_name','address','call_number', 'kakao_id']] = ["진진", "서울 마포구 월드컵북로1길 60", "070-5035-8878", 1767349854]
restaurant.loc[60, ['kakao_name','address','call_number', 'kakao_id']] = ["오레노라멘 합정본점", "서울 마포구 독막로8길 16", "02-322-3539", 1916682638]
restaurant.loc[68, ['kakao_name','address','call_number', 'kakao_id']] = ["명동교자 본점", "서울 중구 명동10길 29", "02-776-5348", 10332413]
restaurant.loc[78, ['kakao_name','address','call_number', 'kakao_id']] = ["삼청동수제비", "서울 종로구 삼청로 101-1", "02-735-2965", 13559837]
restaurant.loc[90, ['kakao_name','address','latitude', 'longitude', 'call_number', 'kakao_id']] = ["담택", "서울 마포구 동교로12안길 51", 37.555015, 126.923422, "02-336-0974", 1534943798]
restaurant.loc[105, ['kakao_name','address','call_number', 'kakao_id']] = ["스시마츠모토", "서울 강남구 도산대로75길 24", "02-543-4334", 1784630600]
restaurant.loc[138, ['kakao_name','address','call_number', 'kakao_id']] = ["역전회관", "서울 마포구 토정로37길 47", "02-703-0019", 18279130]
restaurant.loc[176, ['kakao_name','address','call_number', 'kakao_id']] = ["구찌 오스테리아 서울", "서울 강남구 압구정로 462", "02-795-1119", 623581617]
restaurant.loc[182, ['kakao_name','address','call_number', 'kakao_id']] = ["오프닝", "서울 강남구 도산대로34길 22", "02-2088-5844", 2010673091]
restaurant.loc[194, ['kakao_name','address','call_number', 'kakao_id']] = ["정육면체", "서울 서대문구 연세로5다길 22-8", "070-4179-3819", 1705674698]
restaurant.loc[198, ['kakao_name','address','call_number', 'kakao_id']] = ["으뜸이로리바타", "부산 수영구 수영로408번길 20", "010-6640-2884", 1227688599]
restaurant.loc[199, ['kakao_name','address','call_number', 'kakao_id']] = ["오부이용", "서울 성동구 독서당로51길 29-1", "0502-5551-1801", 1715937835]
restaurant.loc[211, ['kakao_name','address','call_number', 'kakao_id']] = ["미미면가 본점", "서울 강남구 강남대로160길 29", "0507-1413-5466", 18324313]
restaurant.loc[223, ['kakao_name','address','call_number', 'kakao_id']] = ["레썽스", "부산 수영구 광남로22번길 17", "0503-7150-1100", 1394878342]
restaurant.loc[244, ['kakao_name','address','call_number', 'kakao_id']] = ["르오뇽", "서울 강남구 도산대로17길 29", "0507-1324-9187", 1243908002]


In [81]:
restaurant.to_excel("D:/Users/Downloads/restaurant.xlsx", index=False)

In [82]:
restaurant.to_csv("D:/Users/Downloads/restaurant.csv", index=False, encoding='utf-8')